In [ ]:
#!/usr/bin/env python
#
# threading
# "basic" example
#

import random
import threading
import time
import logging

logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-10s) %(message)s',
                    )

def worker():
    """thread worker function"""
    t = threading.currentThread()
    pause = random.randint(1,5)
    logging.debug('sleeping %s', pause)
    time.sleep(pause)
    logging.debug('ending')
    return

for i in range(3):
    t = threading.Thread(target=worker)
    t.setDaemon(True)
    t.start()

main_thread = threading.currentThread()
for t in threading.enumerate():
    print t.getName()
    if t is main_thread:
        print "t is main thread"
        continue
    logging.debug('joining %s', t.getName())
    t.join()

In [ ]:
#!/usr/bin/env python
#
# threading
# "advanced" example
#

import threading
import Queue
import time
import logging


# set logging.debug format
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s [%(levelname)s] (%(threadName)s) %(message)s',)


# a worker thread for reading tasks from the 'taskqueue' Queue object, and adding messages to the
# 'logwriter' queue to be written to a logfile
class WorkerThread(threading.Thread):
        def __init__(self, inQueue, outQueue):
                threading.Thread.__init__(self)
                self.inQueue = inQueue
                self.outQueue = outQueue

        def run(self):
                while True:
                        counter = self.inQueue.get()
                        print "process %d" % counter
                        #logging.debug('processing value %d' % counter)
                        #time.sleep(2)
                        self.outQueue.put("Bananas processed this cycle " + str(counter))
                        print "done %d" % counter
                        #logging.debug('value %d processed' % counter)
                        self.inQueue.task_done()



# a worker thread for reading log entries from the 'logqueue' Queue object and writing them
# to a logfile
class LoggingWorkerThread(threading.Thread):
        def __init__(self, queue):
                threading.Thread.__init__(self)
                self.queue = queue

        def run(self):
                while True:
                        nextLog2Write = self.queue.get()
                        logging.debug(nextLog2Write)
                        self.queue.task_done()


# main function - main application logic
def main():
        # instantiate queues to allow data to flow between threads in the application
        taskqueue = Queue.Queue()
        logqueue = Queue.Queue()

        # create logging thread + start it
        logwriter = LoggingWorkerThread(logqueue)
        logwriter.setName('LOGWRITER')
        logwriter.setDaemon(True)
        logwriter.start()



        # create worker threads for processing values (and start them)
        for i in range (3):
                worker = WorkerThread(taskqueue, logqueue)
                worker.setDaemon(True)
                worker.start()
                print "Worker %d created." % i




        # add some integers to the 'taskqueue' Queue object
        for j in range(10):
                taskqueue.put(j)



        # "join" queues - by invoking this method we are committing to waiting for all tasks
        # on a queue to be completed

        taskqueue.join()

        # for some reason the main application won't exit at the moment, without adding a
        # timeout to this 'join'...
        logwriter.join(1)

        print "DONE!"

if __name__ == "__main__":
        main()